In [11]:
from autogen_core.models import ModelInfo
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
from pathlib import Path

model_info = ModelInfo(
    family="r1", vision=False, function_calling=True, json_output=True
)
model_client = OllamaChatCompletionClient(model="qwen3:14b", model_info=model_info)
# 获取当前工作目录
workdir = Path.cwd()

server_path = workdir.parent / "src" / "mcp-demo" / "server.py"
server_params = StdioServerParams(
    command="uv",
    args=["run", str(server_path)]
)
tools = await mcp_server_tools(server_params)

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=tools,
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


async def main() -> None:
    await Console(agent.run_stream(task="深圳的天气适合出游吗"), output_stats=True)
    # Close the connection to the model client.
    await model_client.close()


await main()

---------- TextMessage (user) ----------
深圳的天气适合出游吗
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='0', arguments='{"city": "\\u6df1\\u5733"}', name='get_weather')]
[Prompt tokens: 200, Completion tokens: 131]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='[{"type": "text", "text": "{\\n  \\"temperature\\": 26,\\n  \\"condition\\": \\"\\u5c0f\\u96e8\\"\\n}", "annotations": null}]', name='get_weather', call_id='0', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
<think>
好的，我现在需要处理用户的问题：“深圳的天气适合出游吗”。首先，用户之前已经询问过这个问题，我调用了天气接口获取了深圳的天气情况，返回的数据显示温度是26度，天气是小雨。现在用户可能再次询问或者需要进一步的建议。

首先，我要确认当前的天气数据是否仍然适用。因为天气可能会变化，但假设用户现在再次询问，可能希望得到更详细的建议或者确认之前的天气情况。不过根据之前的回复，温度26度，小雨，这样的天气是否适合出游呢？

接下来，我需要分析天气状况对出游的影响。温度26度属于舒适范围，但小雨可能会影响户外活动。需要考虑用户可能的活动类型，比如是否带伞、是否需要改期等。此外，可能还需要提供一些建议，比如建议携带雨具，选择室内景点，或者推荐雨天适合的活动。

另外，用户可能没有明确说明他们计划去的具体景点，所以需要给出一般性的建议，同时提醒他们注意天气变化。如果有其他天气因素，比如湿度、风速等，可能也需要考虑，但当前数据中